In [1]:
import pandas as pd
from unicodedata import category

df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [28]:
df2 = pd.read_csv('spam2.csv')
y2 = pd.DataFrame([message[message.index('Subject') + 9: ] for message in df2.email], columns=['Message'])
df2.email = y2
df2 = df2.drop(columns=['id', 'email']).rename(columns={'label': 'Category'})
df2 = pd.concat([df2, y2], axis=1)
df2.head()

,Category,Message
0,ham,Regarding Your Recent Inquiry\n\nThank you for...
1,ham,Weekly Newsletter - Latest Updates\n\nPlease f...
2,ham,Important: Software Update Notification\n\nTha...
3,ham,Team Stand-up at 10 AM\n\nPlease find attached...
4,ham,Team Stand-up at 10 AM\n\nHere's your weekly d...


In [5]:
df3 = pd.read_csv('spam3.csv')
df3 = df3.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']).rename(columns={'v1': 'Category', 'v2': 'Message'})
df3.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
df4 = pd.read_csv('spam4.csv')

y4 = pd.DataFrame(['spam' if category == 'spam' else 'ham' for category in df4.type], columns=['Category'])
df4 = df4.drop(columns=['title', 'type']).rename(columns={'text': 'Message'})
df4 = pd.concat([y4, df4], axis=1)
df4

,Category,Message
0,spam,"Hi James,\n\nHave you claim your complimentary..."
1,ham,"\nalt_text\nCongratulations, you just earned\n..."
2,ham,"Here's your GitHub launch code, @Mortyj420!\n ..."
3,ham,"Hello,\n \nThank you for contacting the Virtua..."
4,spam,"Hey Prachanda Rawal,\n\nToday's newsletter is ..."
...,...,...
79,ham,"Dear Maryam, \n\n \n\nI would like to thank yo..."
80,ham,"Dear Customer,\n\nWelcome to Kilimall, Thanks ..."
81,ham,"Dear vladis163rus,\nHere is the Steam Guard co..."
82,ham,View In Browser | Log in\n \n \n\nSkrill logo\...


In [29]:
dataset = pd.concat([df, df2, df3, df4], axis=0)
dataset

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
79,ham,"Dear Maryam, \n\n \n\nI would like to thank yo..."
80,ham,"Dear Customer,\n\nWelcome to Kilimall, Thanks ..."
81,ham,"Dear vladis163rus,\nHere is the Steam Guard co..."
82,ham,View In Browser | Log in\n \n \n\nSkrill logo\...


In [42]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
X = dataset.Message
y = encoder.fit_transform(dataset.Category)
y

array([0, 0, 1, ..., 0, 0, 0])

In [43]:
from SpamClassifier import NBC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(list(X), y, test_size=0.1, random_state=42)

model = NBC()
model.fit(X_train, y_train)

In [44]:
nonspam, spam = model.get_dicts()
count_nonspam, count_spam = model.get_counters()
print(f"nonspam: {nonspam}, \ncounter: {count_nonspam}")
print(f"spam: {spam},\n counter: {count_spam}")

nonspam: {'feedback': 0.30557326871572504, 'request': 0.20533095268460655, 'your': 1.8970794541512561, 'recent': 0.39816349955362834, 'purchase': 0.2027802576202015, 'team': 0.4921566126769545, 'just': 0.2639969391659227, 'reminder': 0.28516770820048465, 'about': 0.32852952429537047, 'project': 0.29052416783573526, 'alpha': 0.18365004463716364, 'meeting': 0.2948603494452238, 'tomorrow': 0.11197551332738172, 'conference': 0.09246269608468308, 'room': 0.10049738553755898, 'please': 0.6023466394592526, 'come': 0.14487947965820686, 'prepared': 0.09271776559112357, 'discuss': 0.09310036985078433, 'roadmap': 0.09220762657824257, 'best': 0.2975385792628491, 'john': 0.09526846065552863, 'stand': 0.20010202780257622, 'here': 0.3185818135441908, 'weekly': 0.19436296390766483, 'dose': 0.09539599540874888, 'news': 0.09705394720061217, 'updates': 0.28988649406963396, 'from': 0.4281341665603877, 'community': 0.09603366917485015, 'this': 0.35250605790077794, 'week': 0.11057263104195893, 'cover': 0.09

In [45]:
model.score(X_test, y_test)

0.9684408855393312

In [46]:
model.predict(["Hey,LED skincare is having a moment. From under eye devices to scalp brushes, more people are bringing spa-level tech into their daily routines.This LED scalp brush is turning heads. Light therapy and gentle massage work together to refresh your scalp and support hair growth.See the full lineup below.GeoThieve Curator"])

[0]

## Using sklearn Bayes

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split


vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)
model2 = MultinomialNB()
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y)

model2.fit(X_train, y_train)

MultinomialNB()

In [48]:
model2.score(X_test, y_test)

0.9760693423779914

In [61]:
test = ['spam spam']
model2.predict(vectorizer.transform(test))

array([1])

In [62]:
model.predict_proba(test)

[[{0: -15.690215373744467, 1: -15.01938816781955}]]

In [63]:
model.predict(test)

[1]

In [52]:
model.save_options()